In [ ]:
%pylab notebook

# LOFAR failure analysis: main notebook
# V.A. Moss (vmoss.astro@gmail.com)
__author__ = "V.A. Moss"
__date__ = "$25-apr-2019 17:00:00$"
__version__ = "1.0"

import os
import sys
import time
rc('text', usetex=False)
rc('font',**{'size':9,'weight':'normal'})
from astropy.io import ascii
import datetime
import operator

In [ ]:
# Initialisation values
csv_name = 'input/Cycle11-Part1.csv'
save_loc = 'results/'

# Which cycle is it?
cyc = 'Cycle 11'
today = datetime.datetime.today().strftime('%d %b %Y')
today_save = str(datetime.datetime.utcnow().date())

# Files to use for historical plotting
files = ['input/Cycle8-Part1.csv',
         'input/Cycle8-Part2.csv',
        'input/Cycle9-Part1.csv',
        'input/Cycle9-Part2.csv',
        'input/Cycle10-Part1.csv',
        'input/Cycle10-Part2.csv',
        'input/Cycle11-Part1.csv']

In [ ]:
d = ascii.read(csv_name)
print (d.keys())

In [ ]:
# Pie chart of project codes
pcodes = {}
unique = (list(set(d['PROJECT CODE'])))
for x in unique:
    xname = '_'.join(x.split('_'))
    pcodes[xname] = list(d['PROJECT CODE']).count(x)
print (pcodes)
print ( 'Total projects: %i' % (sum(list(pcodes.values()))))
print  ('Total unique projects: %i' % len(unique))

sorted_p = sorted(pcodes.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_p:
    values.append(x[1])
    keys.append(x[0])
    
coldict = {}
clist = [cm.Spectral(x) for x in linspace(0,1,len(pcodes))]
clist2 = clist[::-1]
for i in range(0,len(keys)):
    coldict[keys[i]] = clist2[i]

# Make the pie chart
figure(figsize=(5,5))
title('%s failed projects (incidence-weighted): %s' % (cyc,today))
patches, texts, autotexts = pie(values[::-1],colors=clist,wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')
legend(tuple(patches), tuple(keys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.3))
masterpatches,masterkeys = patches,keys
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failedprojects_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)


In [ ]:
# Make the pie chart (weighted by time)
pcodes_time = {}
unique = list(set(d['PROJECT CODE']))
for x in unique:
    xname = '_'.join(x.split('_'))
    
    # Get the amount of time
    mask = (d['PROJECT CODE'] == x)
    dsub = d[mask]
    losttime = sum(dsub['OBSERVING HOURS LOST'])    
    #pcodes_time[xname] = list(d['PROJECT CODE']).count(x)*losttime
    pcodes_time[xname] = losttime
    print( xname,losttime)
    
sorted_ptime = sorted(pcodes_time.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_ptime:
    values.append(x[1])
    keys.append(x[0])
    
# Keep the same colours
clist = []
for x in keys:
    clist.append(coldict[x])
clist = clist[::-1]
    
figure(figsize=(5,5))
title('Failed projects (time weighted)')
title('%s failed projects (time-weighted): %s' % (cyc,today))
patches, texts, autotexts = pie(values[::-1],labeldistance=1.2,colors=clist,wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')#,pctdistance=1.1)
legend(tuple(masterpatches), tuple(masterkeys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.3))
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failedprojects_timeweighted_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Make the pie chart (weighted by category)
catcodes = {}
allcodes = []
for i in range(0,len(d['FAILURE CATEGORY'])):
    print(d['FAILURE CATEGORY'][i])
    cat = [x.strip() for x in d['FAILURE CATEGORY'][i].split(',')]
    for j in cat:
        allcodes.append(j)
unique = list(set(allcodes))
for x in unique:
    catcodes[x] = allcodes.count(x)
    
sorted_cat = sorted(catcodes.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_cat:
    values.append(x[1])
    keys.append(x[0])
    
coldict = {}
clist = [cm.Spectral(x) for x in linspace(0,1,len(catcodes.keys()))]
clist2 = clist[::-1]
for i in range(0,len(keys)):
    print (keys[i],clist2[i])
    coldict[keys[i]] = clist2[i]    

figure(figsize=(5,5))
title('%s failure reasons (incidence-weighted): %s' % (cyc,today))
patches,texts,autotexts = pie(values[::-1],labeldistance=1.2,colors=clist,wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')
legend(tuple(patches), tuple(keys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.1))
masterpatches,masterkeys = patches,keys
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failurereasons_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Make the pie chart (weighted by category and time)
catcodes = {}
cattime = {}
allcodes = []
for i in range(0,len(d['FAILURE CATEGORY'])):
    cat = [x.strip() for x in d['FAILURE CATEGORY'][i].split(',')]
    timelost = d['OBSERVING HOURS LOST'][i]
    divtime = timelost/float(len(cat))
    for j in cat:
        allcodes.append(j)
        
        # Append to time
        if j in cattime.keys():
            cattime[j]+=divtime
        else:
            cattime[j] = divtime
        
unique = list(set(allcodes))
for x in unique:
    tottime = sum(cattime[x])
    catcodes[x] = tottime

sorted_cat = sorted(catcodes.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_cat:
    values.append(x[1])
    keys.append(x[0])  
    
# Keep the same colours
clist = []
for x in keys:
    clist.append(coldict[x])
clist = clist[::-1]
coldictcat = coldict
    
figure(figsize=(5,5))
title('%s failure reasons (time-weighted): %s' % (cyc,today))
patches,texts,autotexts = pie(values[::-1],labeldistance=1.2,colors=clist,wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')
legend(tuple(masterpatches), tuple(masterkeys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.1))
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failurereasons_timeweighted_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Make the pie chart - subcategories
subcats = {}
cattime = {}
allcodes = []
catcodes = {}
for i in range(0,len(d)):
    cat = [x.strip() for x in d['FAILURE CATEGORY'][i].split(',')]
    print(d['SUBCATEGORY'][i])
    subcat = [x.strip() for x in d['SUBCATEGORY'][i].split(',')]
    timelost = d['OBSERVING HOURS LOST'][i]
    divtime = timelost/float(len(subcat))
    for jj in range(0,len(subcat)):
        ocat = cat[jj]
#         if ocat not in ['STATION','SYSTEM','COBALT']:
#             continue
        j = subcat[jj]
        allcodes.append(j)
        
        # Append to time
        if j in cattime.keys():
            cattime[j]+=divtime
        else:
            cattime[j] = divtime
        
unique = list(set(allcodes))
for x in unique:
    tottime = sum(cattime[x])
    catcodes[x] = allcodes.count(x)#*tottime

sorted_cat = sorted(catcodes.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_cat:
    values.append(x[1])
    keys.append(x[0])    
 
print (sorted_cat)

coldict = {}
clist = [cm.Spectral(x) for x in linspace(0,1,len(cattime.keys()))]
clist2 = clist[::-1]
for i in range(0,len(keys)):
    coldict[keys[i]] = clist2[i]
    
print( coldict,clist)
    
figure(figsize=(5,5))
title('%s failure reasons (incidence-weighted): %s' % (cyc,today))
patches,texts,autotexts = pie(values[::-1],labeldistance=1.2,colors=clist,wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')
legend(tuple(patches), tuple(keys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.15))
masterpatches,masterkeys = patches,keys
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failurereasons_subcategories_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Make the pie chart - subcategories
subcats = {}
cattime = {}
allcodes = []
catcodes = {}
for i in range(0,len(d)):
    cat = [x.strip() for x in d['FAILURE CATEGORY'][i].split(',')]
    subcat = [x.strip() for x in d['SUBCATEGORY'][i].split(',')]
    timelost = d['OBSERVING HOURS LOST'][i]
    divtime = timelost/float(len(subcat))
    
    print (cat,divtime,subcat)
    for jj in range(0,len(subcat)):
        ocat = cat[jj]
        print (ocat)
    
#         if ocat not in ['STATION','SYSTEM','COBALT']:
#             continue
        j = subcat[jj]
        allcodes.append(j)
        
        # Append to time
        if j in cattime.keys():
            cattime[j]+=divtime
        else:
            cattime[j] = divtime
        
unique = list(set(allcodes))
for x in unique:
    tottime = sum(cattime[x])
    catcodes[x] = allcodes.count(x)#*tottime

sorted_cat = sorted(cattime.items(), key=operator.itemgetter(1))
values,keys = [],[]
for x in sorted_cat:
    values.append(x[1])
    keys.append(x[0]) 

# Keep the same colours
clist = []
print (coldict)
for x in keys:
    clist.append(coldict[x])
    
print( sum(values),values)
    
figure(figsize=(5,5))
title('%s failure reasons (time-weighted): %s' % (cyc,today))
patches,texts,autotexts = pie(values[::-1],labeldistance=1.2,colors=clist[::-1],wedgeprops = {'linewidth': 1,'edgecolor':'k'},autopct='%1.0f%%')
legend(tuple(masterpatches), tuple(masterkeys[::-1]), loc=8,ncol=3,bbox_to_anchor=(0.5,-0.15))
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_weight('bold')
    autotext.set_size(12)
savefig('%s/%s_failurereasons_subcatergories_time_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)


In [ ]:
# Plot data loss over time
timelostdate = {}

for i in range(0,len(d)):
    dates = d['DATE'][i].split(',')
    timelost = d['OBSERVING HOURS LOST'][i]
    divtime = timelost/float(len(dates))
    for j in dates:
        date = datetime.datetime.strptime(j.strip()+' 12:00','%Y-%m-%d %H:%M')
        if date in timelostdate:
            timelostdate[date]+=divtime
        else:
            timelostdate[date]=divtime

figure(figsize=(12,3))
x,y = [],[]
for i in sorted(timelostdate):
    x.append(i)
    y.append(timelostdate[i])
plot(x,y,'k-')    
scatter(x,y,marker='o',facecolor='r',edgecolor='k',zorder=100)    
xlabel('Date')
ylabel('Time lost (hr)')
grid(True)
title('%s observing hours lost vs. date' % (cyc))
savefig('%s/%s_timelostvsdate_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Cumulative time lost
x = np.array(x)
ycumu = []
for i in range(0,len(x)):
    j = sum(y[0:i+1])
    ycumu.append(j)

allx = x
ally = ycumu    
    
figure(figsize=(12,3))
plot(x,ycumu,'k-')    
scatter(x,ycumu,marker='o',facecolor='r',edgecolor='k',zorder=100)    
xlabel('Date')
ylabel('Cumulative time lost (hr)')
grid(True)
title('%s cumulative observing hours lost vs. date' % (cyc))
savefig('%s/%s_timelostvsdate_cumulative_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
print (sum(d['OBSERVING HOURS LOST']))

In [ ]:
# Plot data loss in particular categories over time
catloss = {'STATION':[],'CEP4':[],'COBALT':[],'SYSTEM':[],'HUMAN':[],'ENVIRONMENT':[]}
alldates = []

for i in range(0,len(d)):
    dates = d['DATE'][i].split(',')
    timelost = d['OBSERVING HOURS LOST'][i]
    category = d['FAILURE CATEGORY'][i].split(',')
    divtime = timelost/float(len(dates))
    for k in category:
        k = k.strip()
        for j in dates:
            date = datetime.datetime.strptime(j.strip()+' 12:00','%Y-%m-%d %H:%M')
            alldates.append(date)
            if k in catloss:
                catloss[k].append([date,divtime])
            else:
                catloss[k] = [[date,divtime]]
            
fig1, axes1 = subplots(nrows=2, ncols=3,sharex=True, sharey=True,figsize=(12,5))
print (catloss.keys())

for i in range(0,len(catloss)):
    key = list(catloss.keys())[i]
    subloss = {}
    subset = catloss[key]
    label = key
    for j in subset:
        if j[0] in subloss:
            subloss[j[0]]+=j[1]
        else:
            subloss[j[0]]=j[1]
    x,y = [],[]
    for k in sorted(subloss):
        x.append(k)
        y.append(subloss[k])
        
    # Cumulative time lost
    xx,yy = int(ceil(i/3.)), i % 3
    if yy == 0:
        yy = 3
    ax = axes1[xx-1,yy-1]  
    x = np.array(x)
    ycumu = []
    for l in range(0,len(x)):
        j = sum(y[0:l+1])
        ycumu.append(j)
    #colr = cm.Spectral(float(i)/(len(catloss)-1))
    ax.grid(True,alpha=0.2)
    ax.plot(allx,ally,'-',color='k',label='All')
    ax.fill_between(allx,y1=0,y2=ally,color='k',alpha=0.2)
    
    # Get colour
    if key in coldictcat.keys():
        colr = coldictcat[key]
    else:
        colr = 'gray'
    
    # Deal with single point failures
    x = list(x)
    if len(x) == 1:
        xnew = []
        ynew = []
        xnew.append(x[0] - datetime.timedelta(days=1))
        xnew.append(x[0])
        xnew.append(x[0] + datetime.timedelta(days=1))
        ynew.append(y[0])
        ynew.append(y[0])
        ynew.append(y[0])
        x = xnew
        ycumu = ynew
        
    ax.plot(x,ycumu,'-',color=colr,label=key)    
    ax.fill_between(x,y1=0,y2=ycumu,color=colr,alpha=0.5)
    ax.set_xlim(min(alldates),max(alldates))
    ax.set_ylim(0,60)
    ax.legend(loc=0)
    fig1.autofmt_xdate()
    
    
#xlabel('Date')
#ylabel('Time lost (hr)')
#title('Cycle 8 observing hours lost vs. date')
savefig('%s/%s_timelostvsdate_categories_%s.png' % (save_loc,''.join(cyc.split()),today_save),bbox_inches='tight',dpi=200)

In [ ]:
# Historical plot
from historical_failures import *
plot_historical_failures(files,save_loc,cyc)